In [1]:
import gcsfs #module for google cloud connection
import os
import allel
import zarr
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn as sns
import h5py

In [3]:
gcs_orig = gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token='cache')
gcs =  gcsfs.GCSFileSystem(project='malariagen-jupyterhub', token=gcs_orig.session.credentials)

In [4]:
phase2path = os.path.join("ag1000g-release", "phase2.AR1", "variation", "main", "zarr", "biallelic", "ag1000g.phase2.ar1.pass.biallelic")

In [10]:
gcsmap2 = gcsfs.mapping.GCSMap(phase2path, gcs=gcs)

In [11]:
phase1path = os.path.join("ag1000g-release", "phase1.AR3", "variation", "main", "zarr", "ag1000g.phase1.ar3.pass.biallelic")

In [12]:
gcsmap1 = gcsfs.mapping.GCSMap(phase1path, gcs=gcs)

In [13]:
calldata_phase1= zarr.Group(gcsmap1, read_only=True)

In [14]:
variants_phase1 = allel.VariantChunkedTable(calldata_phase1["2L"]["variants"], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'numalt'],
                                     index='POS')
variants_phase1

<VariantChunkedTable shape=(8296600,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=174.1M cbytes=-7 cratio=-26075028.6
   values=zarr.hierarchy.Group>

In [15]:
calldata_phase2= zarr.Group(gcsmap2, read_only=True)

In [16]:
variants_phase2 = allel.VariantChunkedTable(calldata_phase2["2L"]["variants"], 
                                     names=['POS', 'REF', 'ALT', 'DP', 'MQ', 'QD', 'numalt'],
                                     index='POS')
variants_phase2

<VariantChunkedTable shape=(8906423,) dtype=[('POS', '<i4'), ('REF', 'S1'), ('ALT', 'S1'), ('DP', '<i4'), ('MQ', '<f4'), ('QD', '<f4'), ('numalt', '<i4')]
   nbytes=186.9M cbytes=-7 cratio=-27991615.1
   values=zarr.hierarchy.Group>

In [29]:
calldata_out= h5py.File('/gcs/phase1.AR3/extras/outgroup_alleles.h5', mode='r')
calldata_out

<HDF5 file "outgroup_alleles.h5" (mode r)>

In [56]:
calldata_chr = calldata_out['2L']['chri']
calldata_chr

<HDF5 dataset "chri": shape (10377280,), type "|S1">

In [58]:
calldata_chr.dtype

dtype('S1')

In [59]:
ac_chri = allel.AlleleCountsArray(calldata_chr)
ac_chri

TypeError: bad dtype kind: expected on of ('u', 'i'); found S

In [49]:
allel.AlleleCountsArray(calldata_out['2L']['chri'])

TypeError: bad dtype kind: expected on of ('u', 'i'); found S

In [25]:
variants_outgroup = allel.VariantChunkedTable(calldata_out["2L"]["variants"])
variants_outgroup

KeyError: "Unable to open object (object 'variants' doesn't exist)"